## Deep Learning With Python: Handwritten Digit Recognition 

We are going to use the MNIST data-set. The MNIST data-set consists of **60,000 training** samples and **10,000 testing** samples of handwritten digit images. The images are of size **28×28 pixels** and the output can lie between **0-9**.

**The task here is to train a model which can accurately identify the digit present on the image**

### Import MNIST data

In [1]:
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


### Parameters and Network Parameters

In [2]:
batch_size = 128
epochs = 10
n_classes = 10 # MNIST total classes (0-9 digits)
input_shape = (28, 28, 1)

### Preprocess the data

In [3]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

y_train = keras.utils.np_utils.to_categorical(y_train, n_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, n_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # dividing all the values by 255 will convert it to range from 0 to 1
x_test /= 255  # dividing all the values by 255 will convert it to range from 0 to 1

print("x_train shape: {}".format(x_train.shape))
print("{} train samples".format(x_train.shape[0]))
print("{} test samples".format(x_test.shape[0]))

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Create model

Now we will create out **CNN model**.

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

2022-03-12 16:44:07.939894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Train the model

In [5]:
model.fit(x_train, y_train, batch_size= batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 243s 515ms/step - loss: 0.1710 - accuracy: 0.9490 - val_loss: 0.0448 - val_accuracy: 0.9860
Epoch 2/10
469/469 [==============================] - 211s 450ms/step - loss: 0.0566 - accuracy: 0.9831 - val_loss: 0.0356 - val_accuracy: 0.9874
Epoch 3/10
469/469 [==============================] - 206s 439ms/step - loss: 0.0406 - accuracy: 0.9873 - val_loss: 0.0262 - val_accuracy: 0.9909
Epoch 4/10
469/469 [==============================] - 220s 469ms/step - loss: 0.0305 - accuracy: 0.9902 - val_loss: 0.0241 - val_accuracy: 0.9926
Epoch 5/10
469/469 [==============================] - 231s 492ms/step - loss: 0.0260 - accuracy: 0.9916 - val_loss: 0.0246 - val_accuracy: 0.9921
Epoch 6/10
469/469 [==============================] - 187s 399ms/step - loss: 0.0218 - accuracy: 0.9931 - val_loss: 0.0277 - val_accuracy: 0.9909
Epoch 7/10
469/469 [==============================] - 1000s 2s/step - loss: 0.0183 - accuracy: 0.9942 - val_loss: 0.0279 - v

### Evaluate the model

The MNIST dataset is well balanced so we can get around 99% accuracy

In [6]:
 score = model.evaluate(x_test, y_test, verbose=0)
 print("Test loss: {}".format(score[0]))
 print("Test accuracy: {}".format(score[1]))

Test loss: 0.02634400874376297
Test accuracy: 0.991599977016449


### Create GUI to predict digits

In [ ]:
import cv2
import numpy as np
from tkinter import *
from PIL import ImageGrab

source=Tk() #Create the main window
source.resizable(0, 0)
source.title("Digit Recognition")
initx, inity= None, None
image_number=0


def clear_source():
    global draw_area
    draw_area.delete("all")#Delete method for cleaning


def activate_event(event):
    global initx, inity
    draw_area.bind('<B1-Motion>', draw_lines)#Session has started and call draw_lines
    initx, inity = event.x, event.y


def draw_lines(event):
    global initx, inity
    x, y = event.x, event.y
    #Do the drawing
    draw_area.create_line((initx, inity, x, y), width=7, fill='black', capstyle=ROUND, smooth=True, splinesteps=12)
    initx, inity = x, y

def Recognize_Digit():
    global image_number
    filename=f'image_{image_number}.png'
    widget=draw_area
    #Get coordinates of canvas.
    x= source.winfo_rootx() + widget.winfo_x()
    y= source.winfo_rooty() + widget.winfo_y()
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    #Get image by using grab() and crop it. Then save it.
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)

    digit = cv2.imread(filename,cv2.IMREAD_COLOR)#Read image
    make_gray=cv2.cvtColor(digit,cv2.COLOR_BGR2GRAY)#Convert in grayscale
    ret,th=cv2.threshold(make_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)#Otsu threshold
    contours=cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]#extracting contours from image

    for cnt in contours:
        #Get bounding box
        x,y,w,h=cv2.boundingRect(cnt)
        #Create Rectangle
        cv2.rectangle(digit,(x,y),(x+w,y+h),(255,0,0),1)
        top=int(0.05*th.shape[0])
        bottom=top
        left=int(0.05*th.shape[1])
        right=left
        #Extract the image ROI
        roi=th[y-top:y+h+bottom,x-left:x+w+right]
        #Resize ROI image
        img=cv2.resize(roi,(28,28),interpolation=cv2.INTER_AREA)
        #Reshape image to standart of our model
        img=img.reshape(1,28,28,1)
        #Normalizing
        img=img/255.0
        prediction=model.predict([img])[0]
        #Get the maximum values
        final=np.argmax(prediction)
        data=str(final)+'  '+str(int(max(prediction)*100))+'%'
        #Draw the screen
        cv2.putText(digit,data,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),1)
    cv2.imshow('digit',digit)#Show the result
    cv2.waitKey(0)

#Creating canvas
draw_area=Canvas(source, width=640, height=480, bg='white')
draw_area.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)
#Mechanism to let you deal with event yourself
draw_area.bind('<Button-1>', activate_event)
#Add buttons and their functions
btn_save=Button(text="Recognize the Digit",fg='black',command=Recognize_Digit)
btn_save.grid(row=2,column=0,pady=1,padx=1)
button_clear=Button(text="Clear Area",fg='black', command=clear_source)
button_clear.grid(row=2,column=1,pady=1,padx=1)

source.mainloop()


